In [1]:
# install praw to examine reddit api
# pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import nltk
import requests
import praw

In [3]:
# initialize the reddit API
user_agent = "SDA Assignment 2.0 by /u/TheToweringOne"
reddit = praw.Reddit(client_id="zVnTv7HbtIfxMkJPN8yKXA", client_secret="Slh9efonAnd_wFD6BFK3nvmQLSubAA", user_agent = user_agent)

In [5]:
# grab posts from the movies subreddit discussion thread for the movie "The Marvels"
post_id = '17rujx8'
submission = reddit.submission(id=post_id)

In [7]:
# Grab all the main comments from the post commenting on the movie "The Marvels"
submission.comments.replace_more(limit=None)
comments = submission.comments.list()

In [8]:
len(comments)

2379

In [6]:
# install vader to analyze sentiment of social media
# pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
# use vader as a sentiment analysis tool
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [15]:
#place the comments in a data frame
data = {'Comment': [comment.body for comment in comments]}
df = pd.DataFrame(data)
print(df)

                                                Comment
0     Fury just chilling up in space as if he didn’t...
1     Felt like the story played fast and loose with...
2     Kamala: Is this the new ipad?\n\nMonica: They ...
3     Monica Rambeau just fuckin loves touching rand...
4     I was very disappointed that the fighters from...
5     MULTIPLE CAMERA SHOTS OF EARTH! No celestial h...
6     That was the most bizarre sequence I've ever s...
7             Did….did the villains rip-off Spaceballs?
8     Welp... didn't expect to see Mega-Maid from Sp...
9     This should have come out a couple movies afte...
10    Marvel finding Iman Vellani is such a miracle....
11    Very messy, just didn’t capture my interest, o...
12           Finally Kate bishop has a movie appearance
13    So Carol Danvers is Disney Princess right? She...
14    Why is every single opinion that isn’t “this i...
15    The dress Carol wears during the dance with Pr...
16    I’m not an MCU person and I haven’t seen o

In [16]:
# make it a csv file if I need to use it again at some point
# Save the DataFrame to a CSV file
df.to_csv('reddit_comments_the_marvels.csv', encoding = 'utf-8', index=False)

In [27]:
# Use Vader to create a dictionary and assess the comments on the reddit article, place them in a variable called results
results = []

for index, row in df.iterrows():
    pol_score = sid.polarity_scores(row['Comment'])
    pol_score['comment'] = row['Comment']
    results.append(pol_score)


In [30]:
# new data frame with compound sentiment value
df2 = pd.DataFrame.from_records(results)
df2.head()

,neg,neu,pos,compound,comment
0,0.286,0.714,0.000,-0.5859,Fury just chilling up in space as if he didn’t...
1,0.043,0.778,0.180,0.7506,Felt like the story played fast and loose with...
2,0.000,0.701,0.299,0.7691,Kamala: Is this the new ipad?\n\nMonica: They ...
3,0.000,0.715,0.285,0.6115,Monica Rambeau just fuckin loves touching rand...
4,0.194,0.630,0.177,0.2748,I was very disappointed that the fighters from...


In [33]:
# Assess compound coloumn and assign negative, positive, or neutral
df2['label'] = 0
df2.loc[df2['compound'] > 0.1, 'label'] = "positive"
df2.loc[df2['compound'] < -0.1, 'label'] = "negative"
df2.loc[(df2['compound'] >= -0.1) & (df2['compound'] <= 0.1), 'label'] = "neutral"
df2.head()

,neg,neu,pos,compound,comment,label
0,0.286,0.714,0.000,-0.5859,Fury just chilling up in space as if he didn’t...,negative
1,0.043,0.778,0.180,0.7506,Felt like the story played fast and loose with...,positive
2,0.000,0.701,0.299,0.7691,Kamala: Is this the new ipad?\n\nMonica: They ...,positive
3,0.000,0.715,0.285,0.6115,Monica Rambeau just fuckin loves touching rand...,positive
4,0.194,0.630,0.177,0.2748,I was very disappointed that the fighters from...,positive


In [34]:
# make another csv file with the sentiment attached
df3 = df2[['compound', 'comment', 'label']]
df3.to_csv('reddit_sentiment_themarvels_comments.csv', encoding = 'utf-8', index=False)

In [35]:
# check the number of positive, negative, and neutral reviews
print(df2['label'].value_counts())

positive    1281
negative     645
neutral      453
Name: label, dtype: int64


In [37]:
# percentage positive, negative, and neutral reviews
df2.label.value_counts(normalize=True) * 100

positive    53.846154
negative    27.112232
neutral     19.041614
Name: label, dtype: float64

In [38]:
# Examples of sentiment for positive, negative, and neutral

print("Positive Comments:")
positive_comments = df2[df2['label'] == 'positive']['comment'].sample(5) # 5 samples grabbed rather than first 5
for comment in positive_comments: # add a 'positive' before each comment to see where the comment ends
    print(f"Positive: {comment}") # to print the entire comment rather than see it partially with ellipses,
print("")


print("Negative Comments:")
negative_comments = df2[df2['label'] == 'negative']['comment'].sample(5)
for comment in negative_comments:
    print(f"Negative: {comment}")
print("")

print("Neutral Comments:")
neutral_comments = df2[df2['label'] == 'neutral']['comment'].sample(5)
for comment in neutral_comments:
    print(f"Neutral: {comment}")


Positive Comments:
Positive: That's such an interesting view, I never realized that. If true you really changed my perspective on movies lol
Positive: Maria's playing a newer superhero called Binary.
Positive: I walked into this expecting not to enjoy it because of the reviews, but damn did I have a good time. 

I loved kamala. 

I enjoyed this so much more than the first one.
Positive: I think it's a bit early to determine if The Marvel's is "objectively bad" or "objectively good" considering the polarization is rooted in criticisms that ignore people's personal taste in favor of social biases. I thought it was entertaining, I go to Marvel movies to be entertained, and I don't grade them on the same scale as an Oscar contender or say, a Darren Aronofsky film.
Positive: Definitely had more heart than Quantumania imo


Negative Comments:
Negative: The real miss here, is not having song planet be their training montage. They all need to use their powers to fight, but never at the same ti

In [46]:
# look for strong cases of polarity percentage >.5 and <-.5
positive_percentage = (df2['compound'] > 0.5).mean()

negative_percentage = (df2['compound'] < -0.5).mean()

print("Positive high polarity", positive_percentage)
print("")
print("negative high polarity", negative_percentage)

Positive high polarity 0.3694829760403531

negative high polarity 0.1471206389239176
